In [ ]:
#|default_exp distributed.models.spark.lgb

# SparkLGBMForecast

> spark LightGBM forecaster

- skip_exec: true
- skip_showdoc: true

Wrapper of `synapse.ml.lightgbm.LightGBMRegressor` that adds an `extract_local_model` method to get a local version of the trained model and broadcast it to the workers.

In [ ]:
#|export
import lightgbm as lgb
try:
    from synapse.ml.lightgbm import LightGBMRegressor
except ModuleNotFoundError:
    import os
    
    if os.getenv('QUARTO_PREVIEW', '0') == '1' or os.getenv('IN_TEST', '0') == '1':
        LightGBMRegressor = object
    else:
        raise

In [ ]:
#|export
class SparkLGBMForecast(LightGBMRegressor):
    def _pre_fit(self, target_col):
        return self.setLabelCol(target_col)
        
    def extract_local_model(self, trained_model):
        model_str = trained_model.getNativeModel()
        local_model = lgb.Booster(model_str=model_str)
        return local_model